In [33]:
import sys
import os

current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

import pandas as pd
import torch
import pickle
from argparse import Namespace

from utils.utilities_DL import get_loss,load_model_and_optimizer,match_period_coverage_with_netmob
from build_inputs.load_subway_in import load_subway_in
from calendar_class import get_time_slots_labels
from constants.paths import FILE_NAME

trial_ids=['netmob_subway_in_calendar_STGCN_FeatureExtractor_ResNetInspired_MSELoss_2024_08_28_06_04_41108',
                 'netmob_subway_in_calendar_STGCN_ImageAvgPooling_MSELoss_2024_08_27_00_16_90667',
                 'netmob_subway_in_STGCN_FeatureExtractor_ResNetInspired_MSELoss_2024_08_23_06_53_46982',
                 'netmob_subway_in_STGCN_ImageAvgPooling_MSELoss_2024_08_24_01_42_17375',
                 'subway_in_calendar_STGCN_MSELoss_2024_08_25_22_56_92429',
                 'subway_in_STGCN_MSELoss_2024_08_25_18_05_25229']

save_path = 'save/K_fold_validation/best_models'

## Find all the fold save associated to a HP-tuning trial 

In [36]:
model_args = pickle.load(open(f'{working_dir}/{save_path}/model_args.pkl','rb'))
trial_id = 'subway_in_STGCN_MSELoss_2024_08_25_18_05_25229'

best_model_names = [name for name in model_args['model'].keys() if trial_id in name]

## Select one best model: 

In [44]:
selected_model_name = best_model_names[0]
args = model_args['model'][selected_model_name]['args']
perf = model_args['model'][selected_model_name]['performance']

args = Namespace(**args)

# Change proportion to test on TestDataSet
args.train_prop = 0.6
args.valid_prop = 0.2
args.test_prop = 0.2

## Print its associated best Metrics: 

In [45]:
import numpy as np 
valid_losses = []
for model_name in best_model_names:
    valid_losses.append(model_args['model'][model_name]['performance']['valid_loss'])
print('Valid losses: ',valid_losses, 'Mean: ',np.mean(valid_losses))

Valid losses:  [0.0017866326783440934, 0.0019028143752366305, 0.0024192986115813254, 0.002741272183255809] Mean:  0.0022125044621044646


## Load the model and its weights:

In [46]:
selected_model_path = f"{working_dir}/{save_path}/{selected_model_name}.pkl"
model_param = torch.load(selected_model_path)

# Load Model:
coverage = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])
dataset,_,_ = load_subway_in(args,coverage)
_,dic_class2rpz,_,_ = get_time_slots_labels(dataset,nb_class = [0,1,2,3])
loss_function = get_loss(args)
model,optimizer,scheduler = load_model_and_optimizer(args,dic_class2rpz)

# Associate its weights: 
model.load_state_dict(model_param['state_dict'])

Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00

Init Subway-In Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 


 Tackling Training Set

 Tackling Validation Set

 Tackling Testing Set

 Tackling Training Set

 Tackling Validation Set

 Tackling Testing Set

U size:  torch.Size([6719, 40, 7]) Utarget size:  torch.Size([6719, 40, 1])
U_train size:  torch.Size([3912, 40, 7]) Utarget_train size:  torch.Size([3912, 40, 1])
U_valid size:  torch.Size([1304, 40, 7]) Utarget_valid size:  torch.Size([1304, 40, 1])
U_test size:  torch.Size([1310, 40, 7]) Utarget_test size:  torch.Size([1310, 40, 1])
U_train min:  tensor(0.) U_train max:  tensor(1.)
U_valid min:  tensor(0.) U_valid max:  tensor(1.7234)
U_test min:  tensor(0.) U_test max:  tensor(2.2126)
model size: 0.001GB
number of total parameters: 249889
number of trainable parameters: 249889


<All keys matched successfully>